CLINICALBERT + SVM

In [ ]:
# Import libraries
import os
import torch
import numpy as np
import pandas as pd
import warnings
from transformers import AutoTokenizer, AutoModel, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from torch.utils.data import Dataset

# Disable wandb and suppress warnings
os.environ["WANDB_DISABLED"] = "true"
warnings.filterwarnings("ignore")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

# Load tokenizer and base model
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model_name = "emilyalsentzer/Bio_ClinicalBERT"
model = AutoModel.from_pretrained(model_name).to(device)

# Load and preprocess dataset
file_path = "/content/drive/MyDrive/textual_data/TEXTUAL_DATA/processed_disorder_symptoms.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.lower()

# Map frequency and label
frequency_mapping = {
    "Very frequent (99-80%)": 4,
    "Frequent (79-30%)": 3,
    "Occasional (29-5%)": 2,
    "Very rare (<4-1%)": 1
}
df["frequency"] = df["frequency"].map(frequency_mapping)
label_mapping = {label: idx for idx, label in enumerate(df["disorder"].unique())}
df["label"] = df["disorder"].map(label_mapping)

# Dataset class
class SymptomDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
        return {**{k: v.squeeze() for k, v in encoding.items()}, "labels": torch.tensor(label, dtype=torch.long)}

# Train-validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["symptom"].tolist(), df["label"].tolist(), test_size=0.1, random_state=42
)

train_dataset = SymptomDataset(train_texts, train_labels)
val_dataset = SymptomDataset(val_texts, val_labels)

# Load classification model
fine_tuned_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_mapping)).to(device)

# Training arguments
training_args = TrainingArguments(
    output_dir="./clinicalbert_model",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    lr_scheduler_type="linear",
)

# Trainer setup
trainer = Trainer(
    model=fine_tuned_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train model
trainer.train()

# Get embeddings
def get_fine_tuned_bert_embeddings(text_list):
    inputs = tokenizer(text_list, padding=True, truncation=True, return_tensors="pt", max_length=256).to(device)
    with torch.no_grad():
        outputs = fine_tuned_model.bert(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :].to("cpu")
    mean_pooling = torch.mean(outputs.last_hidden_state, dim=1).to("cpu")
    combined_embedding = torch.cat((cls_embedding, mean_pooling), dim=1)
    return combined_embedding.numpy()

# Compute embeddings
symptom_embeddings = np.array([get_fine_tuned_bert_embeddings([symptom])[0] for symptom in df["symptom"]])

# Combine features
X = np.hstack((symptom_embeddings, df["frequency"].values.reshape(-1, 1)))
y = df["disorder"]

# Apply SMOTE
smote = SMOTE(random_state=42, k_neighbors=3)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Normalize features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
svm_model = SVC(kernel="rbf", C=50, gamma="scale", class_weight="balanced", probability=True, random_state=42)
svm_model.fit(X_train, y_train)

# Evaluate
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Fine-Tuned ClinicalBERT + SVM Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

# Prediction function
def predict_disease(symptom_text, frequency_text):
    frequency_value = frequency_mapping.get(frequency_text, 1)
    symptom_vector = get_fine_tuned_bert_embeddings([symptom_text])
    input_features = np.hstack((symptom_vector, np.array([[frequency_value]])))
    input_features = scaler.transform(input_features)
    predicted_disorder = svm_model.predict(input_features)[0]
    return predicted_disorder

# Test prediction
test_symptom = "Short neck"
test_frequency = "Very frequent (99-80%)"
predicted_disease = predict_disease(test_symptom, test_frequency)
print(f"🩺 Predicted Disease: {predicted_disease}")
